In [52]:
import numpy as np
import pandas as pd
import pystan
from scipy.stats import norm, multivariate_normal, invwishart, invgamma
from statsmodels.tsa.stattools import acf
import datetime
import sys
import os

from codebase.plot import * 
from codebase.file_utils import save_obj, load_obj
from codebase.post_process import * 

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
np.random.seed(121)
nsim_data = 200
J = 6
K = 2
beta = np.array([[1,0], [-2, 2],[-1,-1],[2,1], [3,-1], [1,-1]], dtype=float)
mu = np.array([1,2,.3,-.8, 1, -1.4])
sigma = np.array([1,2,.3,.8, 1, 1.4])
Sigma = np.diag(sigma**2)
uu = multivariate_normal.rvs(mean = np.zeros(J), cov = Sigma, size=nsim_data)
zz = multivariate_normal.rvs(mean = np.zeros(K), cov=np.eye(K), size=nsim_data)
yy = mu + zz@beta.T + uu


In [58]:
data = dict()
data['N'] = nsim_data
data['K'] = K
data['J'] = J
data['Sigma'] = Sigma
data['sigma'] = sigma
data['y'] = yy
data['beta'] = beta
data['u'] = uu
data['mu'] = mu
data['Omega'] = data['beta']@data['beta'].T + data['Sigma']

In [4]:
stan_data = dict(N = data['N'], K = data['K'], J = data['J'], yy = data['y'])

In [5]:
with open('./codebase/stan_code/cont_marg.stan', 'r') as file:
    model_code = file.read()


In [6]:
sm = pystan.StanModel(model_code=model_code, verbose=False)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_5ce9a7c63575e3dcf75da5bd59ba59bf NOW.
INFO:pystan:OS: darwin, Python: 3.6.4 | packaged by conda-forge | (default, Dec 23 2017, 16:54:01) 
[GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)], Cython 0.29.5


Compiling /var/folders/9j/qb9gcwnj2lnb77886qjkmnfm0000gn/T/tmptzwzu6jr/stanfit4anon_model_5ce9a7c63575e3dcf75da5bd59ba59bf_6818625680389804035.pyx because it changed.
[1/1] Cythonizing /var/folders/9j/qb9gcwnj2lnb77886qjkmnfm0000gn/T/tmptzwzu6jr/stanfit4anon_model_5ce9a7c63575e3dcf75da5bd59ba59bf_6818625680389804035.pyx
building 'stanfit4anon_model_5ce9a7c63575e3dcf75da5bd59ba59bf_6818625680389804035' extension
creating /var/folders/9j/qb9gcwnj2lnb77886qjkmnfm0000gn/T/tmptzwzu6jr/var
creating /var/folders/9j/qb9gcwnj2lnb77886qjkmnfm0000gn/T/tmptzwzu6jr/var/folders
creating /var/folders/9j/qb9gcwnj2lnb77886qjkmnfm0000gn/T/tmptzwzu6jr/var/folders/9j
creating /var/folders/9j/qb9gcwnj2lnb77886qjkmnfm0000gn/T/tmptzwzu6jr/var/folders/9j/qb9gcwnj2lnb77886qjkmnfm0000gn
creating /var/folders/9j/qb9gcwnj2lnb77886qjkmnfm0000gn/T/tmptzwzu6jr/var/folders/9j/qb9gcwnj2lnb77886qjkmnfm0000gn/T
creating /var/folders/9j/qb9gcwnj2lnb77886qjkmnfm0000gn/T/tmptzwzu6jr/var/folders/9j/qb9gcwnj2lnb77886qjkmnfm0

In [7]:
nowstr = datetime.datetime.now().strftime('%Y%m%d_%H%M%S_') # ISO 8601 format
task_id = 'NN_marg'
log_dir =  "./log/"+nowstr+"%s/" % task_id

In [8]:
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [9]:
num_chains = 1
num_samples = 1000
num_warmup = 1000
num_iter = num_samples + num_warmup

In [10]:
fit = sm.sampling(data=stan_data, iter=num_iter, chains=num_chains)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [11]:
save_obj(sm, 'sm', log_dir)
save_obj(fit, 'fit', log_dir)


/Users/itemgmt/myPhD/factor-analysis/src/codebase/file_utils.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


In [57]:
param_names = ['Omega', 'uu', 'beta', 'mu', 'sigma']

stan_samples= fit.extract(permuted=False, pars=param_names)  # return a dictionary of arrays

if num_chains ==1:
    ps = dict()
    for name in param_names:
        ps[name] = np.squeeze(stan_samples[name])
else: 
    ps = stan_samples

In [24]:
gamma = np.empty_like(ps['beta'])
for i in range(num_samples):
    e_set = get_topn_eig(ps['beta'][i] @ ps['beta'][i].T,
                         data['K'])
    gamma[i] = e_set['P']
ps['gamma']= gamma

In [16]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
    for j in range(data['J']):
            plots.append(plot_trace(ps['Omega'][:,i,j],
                     true_value=data['Omega'][i,j],
                     title = 'Posterior distribution for Omega(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_0_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_0_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_0_comma_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_0_comma_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_0_comma_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_0_comma_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_1_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_1_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_1_comma_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_1_comma_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_1_comma_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_1_comma_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_2_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_2_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_2_comma_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_2_comma_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_2_comma_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_2_comma_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_3_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_3_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_3_comma_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_3_comma_3_right_parenthesis :Overlay
      .C

In [20]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
    for j in range(data['J']):
            plots.append(plot_trace(acf(ps['Omega'][:,i,j]),
                     title = 'Autocorrelation of Omega(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_5_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_5_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_5_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_5_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_5_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_5_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_5_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_5_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_5_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_5_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_5_comma_5_right_parenthesis :Curve   [x]   (y)

In [17]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
            plots.append(plot_trace(ps['mu'][:,i],
                     true_value=data['mu'][i],
                     title = 'Posterior distribution for mu(%s)'%(i)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)

In [21]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
            plots.append(plot_trace(acf(ps['mu'][:,i]),
                     title = 'Autocorrelation of mu(%s)'%(i)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Autocorrelation_of_mu_left_parenthesis_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_mu_left_parenthesis_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_mu_left_parenthesis_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_mu_left_parenthesis_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_mu_left_parenthesis_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_mu_left_parenthesis_5_right_parenthesis :Curve   [x]   (y)

In [59]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
            plots.append(plot_trace(ps['sigma'][:,i],
                     true_value=data['sigma'][i],
                     title = 'Posterior distribution for sigma(%s)'%(i)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Overlay.Posterior_distribution_for_sigma_left_parenthesis_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_sigma_left_parenthesis_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_sigma_left_parenthesis_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_sigma_left_parenthesis_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_sigma_left_parenthesis_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_sigma_left_parenthesis_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)

# Find mode of $\gamma$

In [55]:
g = ps['gamma'][:,:,0]
kde_mode(g)

array([ 0.20606366, -0.50715906, -0.15230187,  0.38644587,  0.66291572,
        0.29721026])

In [25]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
    for j in range(data['K']):
            plots.append(plot_trace(ps['gamma'][:,i,j],
                     title = 'Posterior distribution for gamma(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_0_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_0_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_1_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_1_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_2_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_2_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_3_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_3_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_4_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_4_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_5_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_5_comma_1_right_parenthesis :Curve   [x]   (y)

In [26]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
    for j in range(data['K']):
            plots.append(plot_trace(acf(ps['gamma'][:,i,j]),
                     title = 'Autocorrelation of gamma(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Autocorrelation_of_gamma_left_parenthesis_0_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_0_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_1_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_1_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_2_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_2_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_3_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_3_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_4_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_4_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_5_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_5_comma_1_right_parenthesis :Curve   [x]   (y)